# GEE Aster Preprocessing

## Step 1:

## Step 2: Installation


In [ ]:
%pip install git+https://github.com/Mining-for-the-Future/ASTER_preprocessing.git

## Step 2: Importing Libraries

In [2]:
import ASTER_preprocessing

In [3]:
import ee

In [ ]:
%pip install geemap

In [5]:
import geemap

In [ ]:
from ASTER_preprocessing import preprocessing, data_conversion, masks

## Step 3: Google Earth Engine Authentication

In [8]:
ee.Authenticate()
ee.Initialize(project='my-project')

True

## Step 4: Preprocess ASTER Imagery Using a Drawn Polygon

In [ ]:
# Create an interactive map
Map = geemap.Map()

# Add drawing tools to the map
Map.addDrawingTools()

# Display the map
Map

In [ ]:
# Get the last drawn feature
geometry = Map.draw_last_feature

# Check if a geometry was drawn and it's a Polygon
if geometry is not None and geometry.geometry().type().getInfo() == 'Polygon':
    # Run the preprocessing
    result = preprocessing.aster_preprocessing(geometry.geometry())
    # `result` is a dictionary containing the preprocessed image and metadata
else:
    print("Please draw a polygon on the map.")